# ⚡ Eletroniquiz - Versão Aprimorada
Agora com contagem de erros, botões menores e opções de resposta em minúsculas (a, b, c...)

🎮 **Regras do jogo:**
- ✅ Acertos: **+5 pontos**
- ❌ Erros: **-3 pontos**
- ⏭️ Pulos: **-1 ponto** (2 pulos iniciais)
- 🎁 A cada 10 acertos consecutivos: **+1 pulo extra**
- 🔁 Reforço com perguntas puladas ou erradas no final
- 🎯 Objetivo: **Alcançar 100 pontos!**

## 📌 Como usar:
1. Faça upload do arquivo `perguntas_chatbot_niveis_completas.json`
2. Execute a célula abaixo para iniciar
3. Clique nos **botões menores à esquerda** para responder 🙂

In [ ]:
import json
import random
from IPython.display import display, clear_output
import ipywidgets as widgets

# Variáveis principais
score = 0
pulos = 2
acertos_consecutivos = 0
perguntas_feitas = 0
erros_totais = 0
total_questoes = 40
pendentes = []

# Carrega perguntas
with open("perguntas_chatbot_niveis_completas.json", "r", encoding="utf-8") as f:
    perguntas_por_nivel = json.load(f)

# Monta lista embaralhada
todas_perguntas = []
for nivel in ["1", "2", "3"]:
    todas_perguntas.extend(perguntas_por_nivel[nivel])
random.shuffle(todas_perguntas)

# Widget de saída
output = widgets.Output()

# Barra de status
def barra_status():
    blocos = score // 5
    return f"📊 SCORE: [{'█'*blocos}{'-'*(20 - blocos)}] {score}/100 | 🌀 PULOS: {pulos} | ✅ ACERTOS: {acertos_consecutivos} | ❌ ERROS: {erros_totais}"

# Exibe pergunta com botões
def mostrar_pergunta(pergunta):
    global score, pulos, acertos_consecutivos, perguntas_feitas, erros_totais

    clear_output(wait=True)
    output.clear_output(wait=True)

    print(f"🔧 PERGUNTA {perguntas_feitas + 1}/{total_questoes}")
    print(barra_status())
    print(f"❓ {pergunta['pergunta']}")

    botoes = []
    for opcao in pergunta['opcoes']:
        label = opcao.lower()
        btn = widgets.Button(description=label, layout=widgets.Layout(width='250px', align_self='flex-start'))
        def ao_clicar(b, escolha=label[0], correta=pergunta['resposta'].lower(), p=pergunta):
            global score, acertos_consecutivos, pulos, perguntas_feitas, erros_totais
            if escolha == correta:
                score += 5
                acertos_consecutivos += 1
                print("✅ Correto! +5 pontos.")
            else:
                score = max(0, score - 3)
                acertos_consecutivos = 0
                erros_totais += 1
                pendentes.append(p)
                print(f"❌ Errado! -3 pontos. Resposta correta era: {correta}")
            if acertos_consecutivos > 0 and acertos_consecutivos % 10 == 0:
                pulos += 1
                print("🎁 Bônus: +1 pulo por 10 acertos consecutivos!")
            perguntas_feitas += 1
            proxima()

        btn.on_click(ao_clicar)
        botoes.append(btn)

    botao_pular = widgets.Button(description="⏭️ PULAR", button_style='warning',
                                 layout=widgets.Layout(width='250px', align_self='flex-start'))
    def pular(b):
        global pulos, score, perguntas_feitas
        if pulos > 0:
            pulos -= 1
            score = max(0, score - 1)
            pendentes.append(pergunta)
            print("⏭️ Pergunta pulada. -1 ponto.")
            perguntas_feitas += 1
            proxima()
        else:
            print("🚫 Sem pulos restantes.")
    botao_pular.on_click(pular)

    display(widgets.VBox(botoes + [botao_pular]))

# Próxima pergunta
def proxima():
    if perguntas_feitas < total_questoes and score < 100:
        mostrar_pergunta(todas_perguntas[perguntas_feitas])
    else:
        fim()

# Fim da sessão
def fim():
    clear_output()
    print("🏁 FIM DA SESSÃO!")
    print(barra_status())
    if score >= 100:
        print("🌟 PARABÉNS! Você atingiu o nível mestre!")
    else:
        print("💡 Continue tentando! Reveja as perguntas reforçadas.")

# Início
proxima()
